In [1]:
from typing import List
import torch
from torch import Tensor

/Users/mikeheddes/Developer/cs299/torchhd/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def biggest_power_two(n):
    """Returns the biggest power of two <= n"""
    # if n is a power of two simply return it
    if not (n & (n - 1)):
        return n

    # else set only the most significant bit
    return int("1" + (len(bin(n)) - 3) * "0", 2)

In [5]:
class VSA_Model(Tensor):
    @classmethod
    def random_hv(
        cls,
        num_embeddings: int,
        embedding_dim: int,
        *,
        sparsity=0.5,
        generator=None,
        dtype=None,
        device=None,
        requires_grad=False,
    ):
        raise NotImplementedError

    def bundle(self, other: "VSA_Model") -> "VSA_Model":
        raise NotImplementedError

    def multibundle(self) -> "VSA_Model":
        if self.dim() < 2:
            class_name = self.__class__.__name__
            raise RuntimeError(
                f"{class_name} needs to have at least two dimensions for multibundle, got size: {tuple(self.shape)}"
            )

        n = self.size(-2)
        if n == 1:
            return self.unsqueeze(-2)

        tensors: List[VSA_Model] = torch.unbind(self, dim=-2)
        print(type(tensors[0]))

        output = tensors[0].bundle(tensors[1])
        for i in range(2, n):
            output = output.bundle(tensors[i])

        return output

    def bind(self, other: "VSA_Model") -> "VSA_Model":
        raise NotImplementedError

    def multibind(self) -> "VSA_Model":
        if self.dim() < 2:
            class_name = self.__class__.__name__
            raise RuntimeError(
                f"{class_name} data needs to have at least two dimensions for multibind, got size: {tuple(self.shape)}"
            )

        n = self.size(-2)
        if n == 1:
            return self.unsqueeze(-2)

        tensors: List[VSA_Model] = torch.unbind(self, dim=-2)

        output = tensors[0].bind(tensors[1])
        for i in range(2, n):
            output = output.bind(tensors[i])

        return output

    def permute(self, n: int = 1) -> "VSA_Model":
        raise NotImplementedError


# class MAPTensor(HyperTensor):
#     def bundle(self, other: 'MAPTensor') -> 'MAPTensor':
#         return self + other

#     def multibundle(self) -> 'MAPTensor':
#         return self.sum(dim=-2)

#     def difference(self, other: 'MAPTensor') -> 'MAPTensor':
#         return self - other

#     def bind(self, other: 'MAPTensor') -> 'MAPTensor':
#         return self * other

#     def multibind(self) -> 'MAPTensor':
#         return self.prod(dim=-2)

#     def unbind(self, other: 'MAPTensor') -> 'MAPTensor':
#         return self * other

#     def permute(self, n: int = 1) -> 'MAPTensor':
#         return self.roll(shifts=n, dim=-1)


# class BSCTensor(HyperTensor):
#     def bundle(self, other: Tensor, *, generator: torch.Generator = None) -> Tensor:
#         tiebreaker = torch.empty_like(input)
#         tiebreaker.bernoulli_(0.5, generator=generator)

#         is_majority = self == other
#         return self.where(is_majority, tiebreaker)

#     def multibundle(self, *, generator:torch.Generator = None) -> Tensor:
#         if self.dim() < 2:
#             class_name = self.__class__.__name__
#             raise RuntimeError(f"{class_name} data needs to have at least two dimensions for multibind, got size: {tuple(self.shape)}")

#         n = self.size(-2)

#         count = self.sum(dim=-2, dtype=torch.long)

#         # add a tiebreaker when there are an even number of hvs
#         if n % 2 == 0:
#             tiebreaker = torch.empty_like(count)
#             tiebreaker.bernoulli_(0.5, generator=generator)
#             count += tiebreaker
#             n += 1

#         threshold = n // 2
#         return count > threshold

#     def difference(self, other: Tensor) -> Tensor:
#         return self.logical_and(~other)

#     def bind(self, other: Tensor) -> Tensor:
#         return self.logical_xor(other)

#     def multibind(self) -> Tensor:
#         if self.dim() < 2:
#             class_name = self.__class__.__name__
#             raise RuntimeError(f"{class_name} data needs to have at least two dimensions for multibind, got size: {tuple(self.shape)}")

#         n = self.size(-2)
#         n_ = biggest_power_two(n)
#         output = self[..., :n_, :]

#         # parallelize many XORs in a hierarchical manner
#         # for larger batches this is significantly faster
#         while output.size(-2) > 1:
#             output = torch.logical_xor(output[..., 0::2, :], output[..., 1::2, :])

#         output = output.squeeze(-2)

#         # TODO: as an optimization we could also perform the hierarchical XOR
#         # on the leftovers in a recursive fashion
#         leftovers = torch.unbind(self[..., n_:, :], -2)
#         for i in range(n - n_):
#             output = torch.logical_xor(output, leftovers[i])

#         return output

#     def unbind(self, other: Tensor) -> Tensor:
#         return self.logical_xor(other)

#     def permute(self, n: int = 1) -> Tensor:
#         return self.roll(shifts=n, dim=-1)


# if __name__ == "__main__":
#     x = HyperTensor(torch.randn(2, 6))
#     print(x)
#     print(torch.add(x, 1))
#     x.multibundle()


In [18]:
from enum import Enum

from torch import BoolTensor


class vsa_model(Enum):
    BSC = 1
    MAP = 2
    HRR = 3
    FHRR = 4


class BSCTensor(Tensor):
    def bundle(self, other: 'BSCTensor', *, generator: torch.Generator = None) -> 'BSCTensor':
        tiebreaker = torch.empty_like(self)
        tiebreaker.bernoulli_(0.5, generator=generator)

        is_majority = self == other
        return self.where(is_majority, tiebreaker)

    def bind(self, other: 'BSCTensor') -> 'BSCTensor':
        return self.logical_xor(other)

    def permute(self, n: int = 1) -> 'BSCTensor':
        return self.roll(shifts=n, dim=-1)

class MAPTensor(Tensor):
    def bundle(self, other: 'MAPTensor') -> 'MAPTensor':
        return self.add(other)

    def bind(self, other: 'MAPTensor') -> 'MAPTensor':
        return self.mul(other)

    def permute(self, n: int = 1) -> 'MAPTensor':
        return self.roll(shifts=n, dim=-1)



def random_hv(a, b, model: vsa_model = vsa_model.BSC) -> Tensor:
    result = torch.empty(a, b, dtype=torch.bool).bernoulli_()

    if model == vsa_model.BSC:
        return result.as_subclass(BSCTensor)

    elif model == vsa_model.MAP:
        result = torch.where(result, -1, +1)
        return result.as_subclass(MAPTensor)

    


hv = random_hv(2, 5)
print(hv[0].bind(hv[1]), hv[0].bundle(hv[1]))

hv = random_hv(2, 5, model=vsa_model.MAP)
print(hv[0].bind(hv[1]), hv[0].bundle(hv[1]))

tensor([False,  True, False,  True, False]) tensor([False,  True, False, False, False])
tensor([-1,  1,  1,  1, -1]) tensor([ 0, -2, -2, -2,  0])


In [28]:
class BSC(Tensor):
    @classmethod
    def random_hv(cls, n: int, d: int, dtype=torch.bool):
        result = torch.empty(n, d, dtype=dtype)
        result.bernoulli_()
        return result.as_subclass(cls)

    def bundle(self, other: "BSC", *, generator: torch.Generator = None) -> "BSC":
        tiebreaker = torch.empty_like(self)
        tiebreaker.bernoulli_(0.5, generator=generator)

        is_majority = self == other
        return self.where(is_majority, tiebreaker)

    def bind(self, other: "BSC") -> "BSC":
        return self.logical_xor(other)

    def permute(self, n: int = 1) -> "BSC":
        return self.roll(shifts=n, dim=-1)


class MAP(Tensor):
    @classmethod
    def random_hv(cls, n: int, d: int):
        result = torch.empty(n, d, dtype=torch.bool)
        result.bernoulli_()
        result = torch.where(result, -1, +1)
        return result.as_subclass(cls)

    def bundle(self, other: "MAPTensor") -> "MAPTensor":
        return self.add(other)

    def bind(self, other: "MAPTensor") -> "MAPTensor":
        return self.mul(other)

    def permute(self, n: int = 1) -> "MAPTensor":
        return self.roll(shifts=n, dim=-1)


def random_hv(*args, model=BSC) -> Tensor:
    return model.random_hv(*args)

def bundle(input, other, model=BSC) -> Tensor:
    return input.bundle(other)


hv = random_hv(2, 5)
print(hv[0].bind(hv[1]), hv[0].bundle(hv[1]))

hv = random_hv(2, 5, model=MAP)
print(hv[0].bind(hv[1]), hv[0].bundle(hv[1]))


tensor([ True, False, False, False, False]) tensor([False, False,  True, False, False])
tensor([-1,  1,  1, -1, -1]) tensor([ 0,  2, -2,  0,  0])


In [1]:
import torchhd, torch

hv = torchhd.BSC.random_hv(3, 5)
hv[0].bind(hv[1])

/Users/mikeheddes/Developer/cs299/torchhd/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([ True, False, False, False,  True])

In [2]:
hv = torchhd.MAP.random_hv(3, 5)
hv[0].bind(hv[1])

tensor([ 1., -1.,  1., -1., -1.])

In [3]:
hv = torchhd.HRR.random_hv(3, 5)
hv[0].bind(hv[1])

tensor([ 0.0022,  0.0747,  0.0234, -0.0561,  0.0073])

In [4]:
hv = torchhd.FHRR.random_hv(3, 5)
hv[0].bind(hv[1])

tensor([ 0.6735-0.7391j, -0.3653-0.9309j,  0.0053-1.0000j,  0.8277+0.5612j,
        -0.4813-0.8766j])